In [1]:
BUCKET_NAME="news-classification-2020-mlengine"
REGION="us-central1"

MODEL_DIR="gs://news-classification-2020-mlengine/"
MODEL_NAME="news_classification"
VERSION_NAME="v1"
FRAMEWORK="TENSORFLOW"
MACHINE_TYPE="n1-standard-2"

SAVED_MODEL_DIR="saved_model"
LATEST_MODEL_DIR=!(ls ./$SAVED_MODEL_DIR | tail -1)
LATEST_MODEL_DIR=LATEST_MODEL_DIR[0]

In [ ]:
# エクスポートしたモデルを確認
!saved_model_cli show --all --dir $SAVED_MODEL_DIR/$LATEST_MODEL_DIR

In [ ]:
# Cloud Storage バケットの作成
!gsutil mb -l $REGION gs://$BUCKET_NAME

In [ ]:
# エクスポートしたモデルを Cloud Storage にアップロードする
!gsutil cp -r "$SAVED_MODEL_DIR/$LATEST_MODEL_DIR/*" gs://$BUCKET_NAME

In [ ]:
# ローカル予測でモデルをテストする（JSON）
!gcloud ai-platform local predict --model-dir gs://$BUCKET_NAME \
  --json-instances input.json \
  --framework tensorflow \
  --signature-name serving_default

In [ ]:
# models create
!gcloud ai-platform models create $MODEL_NAME

In [ ]:
# versions create（注意ベータ版を使うこと！）
!gcloud beta ai-platform versions create $VERSION_NAME \
  --model $MODEL_NAME \
  --origin $MODEL_DIR \
  --runtime-version=1.14 \
  --framework $FRAMEWORK \
  --python-version=3.5 \
  --machine-type $MACHINE_TYPE

In [ ]:
# 新しいバージョンに関する情報を取得
!gcloud ai-platform versions describe $VERSION_NAME \
  --model $MODEL_NAME

In [2]:
# 予測のリクエスト
!gcloud ai-platform predict \
  --model $MODEL_NAME \
  --version $VERSION_NAME \
  --json-instances input.json

[[0.00688816654, 0.335943282, 0.0355736651, 0.0707206801, 0.276265204, 0.112958521, 0.0690839365, 0.0407407098, 0.0518258326]]
